In [1]:
from mlp import MultiLayerPerceptron
from TwoDimProblem import TwoDimProblem
import numpy as np

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2

# Test MLP from other file

In [2]:
p = TwoDimProblem(value_range=5)
X, y = p.createData(soln_rank=2, noise_frac=0.02, samples=500)

In [4]:

m = MultiLayerPerceptron(
    epochs=150, 
    lr=0.1,
    activation='sigmoid',
    input_layer=2, 
    hidden_layers=[3,2],
    output_layer=1)
    
m.fit(X,y, batch_size=1)
pred = m.predict(X)
p.plotPred(pred, show_correct=True)
m.plot_loss()

  0%|          | 0/150 [00:00<?, ?it/s]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 1d, C), Tuple(slice<a:b>, none))
 
There are 22 candidate implementations:
[1m     - Of which 20 did not match due to:
     Overload of function 'getitem': File: <numerous>: Line N/A.
       With argument(s): '(array(float64, 1d, C), Tuple(slice<a:b>, none))':[0m
[1m      No match.[0m
[1m     - Of which 2 did not match due to:
     Overload in function 'GetItemBuffer.generic': File: numba\core\typing\arraydecl.py: Line 166.
       With argument(s): '(array(float64, 1d, C), Tuple(slice<a:b>, none))':[0m
[1m      Rejected as the implementation raised a specific error:
        NumbaTypeError: [1munsupported array index type none in Tuple(slice<a:b>, none)[0m[0m
  raised from c:\Users\Matthew\Desktop\Homework\COMP5630\ML-Project\venv\lib\site-packages\numba\core\typing\arraydecl.p